# SetUp Target Server Info

In [84]:
import requests
import json, os

nn_id = 'lstmcrf021'  # put some key value you want to test

url = "{0}:{1}".format(os.environ['HOSTNAME'] , "8000")
print("done")

done


# Net & Version Create

In [85]:
####(1) 네트워크 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/',
                     json={
                         "biz_cate": "MES",
                         "biz_sub_cate": "M60",
                         "nn_title" : "test",
                         "nn_desc": "test desc",
                         "use_flag" : "Y",
                         "dir": "purpose?",
                         "config": "N"
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

####(2) 버전 생성 ####
resp = requests.post('http://' + url + '/api/v1/type/common/target/nninfo/nnid/' + nn_id + '/version/',
                 json={
                     "nn_def_list_info_nn_id": "",
                     "nn_wf_ver_info": "test version info",
                     "condition": "1",
                     "active_flag": "Y"
                 })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


evaluation result : lstmcrf021
evaluation result : lstmcrf021


# Graph Flow 정의 

In [86]:
# Work Flow 틀을 구성하도로고 지시한다. (정해진 틀을 강제로 생성)
resp = requests.post('http://' + url + '/api/v1/type/wf/target/init/mode/simple/' + nn_id +'/wfver/1/',
                     json={
                         "type": 'bilstmcrf_iob'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : bilstmcrf_iob


# 데이터 업로드 

In [87]:
return_dict = {}
return_dict['test'] = open('../../data/ner_train.iob', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/framedata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']


# 데이터 노드 정의 

In [88]:
# (1) Train Data Node 의 속성을 정의
# 어디서 Source 를 어떻게 가지고 올것인지 파라메터 정의 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/iob/prg/source/nnid/'+nn_id+'/ver/1/node/data_node/',
                     json={
                         "type": "local",
                         "source_server": "local",
                         "source_sql": "all",
                     })

# 전처리는 어떤 것을 할지 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/iob/prg/pre/nnid/'+nn_id+'/ver/1/node/data_node/',
                      json={
                          "preprocess":  "None",
                          "word_embed_model" : "w2v0003"
                      })
# 전처리가 완료된 데이터는 어디에 저장을 할지 
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/iob/prg/store/nnid/'+nn_id+'/ver/1/node/data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : /hoya_str_root/lstmcrf021/1/data_node


# 데이터 Feeder 정의

In [89]:
# (2) Network 에 데이터를 Feed하는 Node 의 속성을 정의 
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/iob/net/bilstmcrf/nnid/'+nn_id+'/ver/1/node/pre_feed_train/',
                     json={
                         "seperator" : " ",
                         "col_len" : 2,
                         "word_embed_id" : "",
                         "char_embed_id":  "",
                         "language" : 'kor'
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'char_embed_id': '', 'word_embed_id': '', 'language': 'kor', 'seperator': ' ', 'col_len': 2}


# Bi-Lstm crf 모델 정의

In [90]:
resp = requests.put('http://' + url + '/api/v1/type/wf/state/netconf/detail/bilstmcrf/nnid/' + nn_id + '/ver/1/node/netconf_node/',
                     json={
                        "learning_rate" : 0.01,
                        "iter" : 10,
                        "batch_size" : 10,
                        "examples_to_show" : 10,
                        "n_hidden" : [200, 100] 
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : {'examples_to_show': 10, 'model_path': '/hoya_model_root/lstmcrf021/1/netconf_node', 'iter': 10, 'learning_rate': 0.01, 'batch_size': 10, 'n_hidden': [200, 100]}


# Eval Process 

In [91]:
return_dict = {}
return_dict['test'] = open('../../data/ner_train.iob', 'rb')

resp = requests.post('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     files = return_dict)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 소스 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/raw/prg/source/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                     json={
                         "type": "local",
                         "source_server": "local",
                         "source_sql": "all",
                     })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 전처리 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/raw/prg/pre/nnid/'+nn_id+'/ver/1/node/test_data_node/',
                      json={
                          "preprocess":  "None",
                          "word_embed_model" : "w2v0003"
                      })
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

# 데이터 - 저장 정의
resp = requests.put('http://' + url + '/api/v1/type/wf/state/iobdata/src/local/form/raw/prg/store/nnid/'+nn_id+'/ver/1/node/test_data_node/',)

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))


# Feeder 정의
resp = requests.post('http://' + url + '/api/v1/type/wf/state/pre/detail/feed/src/iob/net/bilstmcrf/nnid/'+nn_id+'/ver/1/node/pre_feed_test/',
                     json={
                         "seperator" : " ",
                         "col_len" : 2,
                         "word_embed_id" : "",
                         "char_embed_id":  "",
                         "language" : 'kor'
                     })
print("evaluation result : {0}".format(data))

node_name = 'eval_node'
resp = requests.put('http://' + url + '/api/v1/type/wf/state/eval/nnid/'+nn_id+'/ver/1/node/eval_node/',
                    json={
                        "type": "category",
                    })

data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : ['1 file upload success']
evaluation result : {'source_parse_type': 'raw', 'source_path': '/hoya_src_root/lstmcrf021/1/test_data_node', 'source_sql': 'all', 'source_server': 'local', 'source_type': 'local'}
evaluation result : None
evaluation result : /hoya_str_root/lstmcrf021/1/test_data_node
evaluation result : /hoya_str_root/lstmcrf021/1/test_data_node
evaluation result : {'type': 'category'}


# Train 

In [99]:
# Run All Workflow
resp = requests.post('http://' + url + '/api/v1/type/runmanager/state/train/nnid/'+nn_id+'/ver/1/')
data = json.loads(resp.json())
print("evaluation result : {0}".format(data))

evaluation result : [None, None, None, None, None, {'labels': ['PERSON', 'O', 'COMMAND', 'TARGET', 'O/'], 'predicts': [[7, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]}]


# Predict

In [81]:
resp = requests.post('http://' + url + '/api/v1/type/service/state/predict/type/bilstmcrf/nnid/'+nn_id+'/ver/active/',
                     json={"input_data" : "이태영의 전화번호 검색" , "num": 0, "clean_ans":False}
                     )
data = json.loads(resp.json())
print("evaluation result(2) : {0}".format(data))

evaluation result(2) : ['B-PERSON', 'B-TARGET', 'O']
